# ReferentialGym: Example Analysis of the Emerging Language

## Data processing:

In [1]:
import pickle
import os 
from tqdm import tqdm
import gc 

def dict_rec_update(d, u):
    if not(isinstance(u,dict)):
        #then d and u are lists:
        for el in u:
            d.append(el)
        return d 
    
    for k in u:
        if k in d:
            d[k] = dict_rec_update(d[k],u[k])
        else:
            d[k] = u[k]
    return d
    
def check_nbr_epoch_in_data(path):
    data = [dict()]
    dicts = [(int(fn[10:]), fn) for fn in os.listdir(path) if  'logs.dict.' in fn]
    dicts.sort(key=lambda x: x[0])
    
    d_path = dicts[-1][1]
    file = open(os.path.join(path, d_path), "rb")
    cdata = pickle.load(file)
    nbr_epoch = len(cdata)
    print("There are {} epochs (testing and training).".format(nbr_epoch))
    file.close()
    del cdata
    gc.collect()
    return nbr_epoch
    
def load_data(path, epoch=0, verbose=True):
    dicts = [(int(fn[10:]), fn) for fn in os.listdir(path) if  'logs.dict.' in fn]
    dicts.sort(key=lambda x: x[0])
    
    data = dict()
    for didx in tqdm(range(len(dicts))):
        d_path = dicts[didx][1]
        file = open(os.path.join(path, d_path), "rb")
        cdata = pickle.load(file)
        
        if verbose:
            print("Looking for epoch {}. Log {} contains {} epochs.".format(epoch,didx,len(cdata)), end='\r')
        if len(cdata) < epoch+1:
            file.close()
            gc.collect()
            continue      
        data = dict_rec_update(data, cdata[epoch])
        file.close()
        gc.collect()
        
    return data

In [2]:
#path = "./MSELoss-1Speaker-None-CulturalObverter0.95-20GPR-S50-full_b128-obs-obverter-tau0-0.2-distr1-stim1-vocab5over20_CIFAR10_pretrained-ResNet18-2"
#path = "./NLLLoss+UsingWIDX+GRU+ClipGrad0.1+ProbOverDistrAndVocab--1Speaker-None-DescriptiveCulturalCategoricalObverter0.95-20GPR-S40-partial_b256-obs-obverter-tau0-0.2-distr0-stim1-vocab5over20_CIFAR10_CNN"
path = "./NLLLoss+UsingWIDX+GRU+Logit4DistrTarNoTarg-1Speaker-None-DescriptiveCulturalObverter0.95-40GPR-S10-partial_b256-obs-obverter-tau0-0.2-distr0-stim1-vocab5over20_CIFAR10_CNN"

In [3]:
nbr_epoch = check_nbr_epoch_in_data(path)
# Retrieving the penultimate test epoch:
epoch = nbr_epoch-3
data = load_data(path, epoch=epoch, verbose=False)

  0%|          | 0/94 [00:00<?, ?it/s]

There are 40 epochs (testing and training).


100%|██████████| 94/94 [02:20<00:00,  1.41s/it]


In [5]:
len(data['os0']['decision'])

46

In [6]:
for agent_id in data:
    for key in data[agent_id]:
        data[agent_id][key] = [ (d[0],d[1]) for d in data[agent_id][key] if d[1] is not None]

In [8]:
for k1 in data:
    it_decisions = [ (s[0],s[1]) for s in data[k1]['decision'] if s[1] is not None]
    it_sentences = [ (s[0],s[1]) for s in data[k1]['sentences_widx'] if s[1] is not None]
    k2 = 'decision'
    print(f"There are {len(it_decisions)} proprer decisions for agent {k1}.")
    k2 = 'sentences_widx'
    print(f"There are {len(it_sentences)} proprer sentences for agent {k1}.")
    #print(f"Final iteration idx: {data[k1][k2][-1][0]}")

There are 0 proprer decisions for agent os0.
There are 11536 proprer sentences for agent os0.
There are 11280 proprer decisions for agent ol0.
There are 0 proprer sentences for agent ol0.


In [9]:
agents = [ values for k,values in data.items()]

In [10]:
agents[0].keys()

dict_keys(['decision', 'sentences_widx', 'sentences_logits', 'sentences', 'features', 'temporal_features'])

## Levenshtein Distance:

In [11]:
# https://www.python-course.eu/levenshtein_distance.php
def compute_levenshtein_distance(s1, s2):
    rows = len(s1)+1
    cols = len(s2)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
    
    # From there, we can compute iteratively how many steps
    # are needed to transform the source prefix (at col) into
    # the target prefix (at row):
    for i in range(1, rows):
        for j in range(1, cols):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = 1
            dist[i][j] = min(dist[i-1][j] + 1,      # deletion
                                 dist[i][j-1] + 1,      # insertion
                                 dist[i-1][j-1] + cost) # substitution
    return float(dist[-1][-1])

In [12]:
compute_levenshtein_distance([0,1,2,3],[0,2,3,4])

2.0

In [13]:
print(agents[0]['sentences_widx'][4][1][0])
print(agents[0]['sentences_widx'][100][1][0])
compute_levenshtein_distance(agents[0]['sentences_widx'][4][1][0],agents[0]['sentences_widx'][100][1][0])

[2]
[3]


1.0

## Cosine Similarity:

In [14]:
import numpy as np
from numpy import linalg as LA

In [15]:
def compute_cosine_sim(v1, v2):
    v1_norm = LA.norm(v1)
    v2_norm = LA.norm(v2)
    cos_sim = np.matmul(v1/v1_norm,(v2/v2_norm).transpose())
    return cos_sim

In [16]:
print(agents[0]['temporal_features'][4][1].shape)
print(agents[0]['temporal_features'][100][1].shape)
compute_cosine_sim(agents[0]['temporal_features'][4][1],agents[0]['temporal_features'][100][1])

(64,)
(64,)


0.5381266

##  Measure of Compositionality as Topographic Similarity (Negative Spearman Correlation):

In [17]:
from scipy.stats import spearmanr

In [20]:
def compute_topographic_similarity(sentences,features,comprange=100):
    levs = []
    for idx1 in tqdm(range(len(sentences))):
        s1 = sentences[idx1]
        tillidx = min(len(sentences)-1,idx1+1+comprange)
        for idx2, s2 in enumerate(sentences[idx1+1:tillidx]): 
            levs.append( compute_levenshtein_distance(s1,s2))
    cossims = []
    for idx1 in tqdm(range(len(features))):
        f1 = features[idx1]
        tillidx = min(len(sentences)-1,idx1+1+comprange)
        for idx2, f2 in enumerate(features[idx1+1:tillidx]): 
            cossims.append( compute_cosine_sim(f1,f2))
    
    rho, p = spearmanr(levs, cossims)
    return -rho, p, levs, cossims  

In [39]:
#%pdb on
nbr_samples = 10000
it_sentences = [ (s[0],s[1]) for s in agents[0]['sentences_widx'][:nbr_samples] if s[1] is not None]
sentences = [s[1] for s in it_sentences]
features = [] 

sentence_idx = 0 
for idx, (it,tf) in enumerate(agents[0]['temporal_features']):
    if len(features) == len(sentences): 
            break
    
    sentence_it = it_sentences[sentence_idx][0]
    if sentence_it == it:
        features.append(tf)
        sentence_idx += 1

comprange = 200
rho, p, levs, cossims = compute_topographic_similarity(sentences=sentences, features=features, comprange=comprange)


  2%|▏         | 200/10000 [00:23<20:33,  7.94it/s]


  4%|▍         | 410/10000 [00:47<20:36,  7.76it/s]


  7%|▋         | 665/10000 [01:12<21:28,  7.24it/s]


  9%|▉         | 935/10000 [01:38<19:57,  7.57it/s]


 12%|█▏        | 1169/10000 [02:03<12:42, 11.58it/s]


...

 96%|█████████▋| 9632/10000 [18:05<00:42,  8.59it/s]


 98%|█████████▊| 9830/10000 [18:28<00:13, 12.33it/s]


 25%|██▍       | 2494/10000 [00:09<00:21, 355.50it/s]


 77%|███████▋  | 7735/10000 [00:25<00:07, 311.15it/s]


100%|██████████| 10000/10000 [00:30<00:00, 325.45it/s]

In [40]:
print("Topographic similarity between the language space and the visual feature space: {} (with p={}).".format(rho,p))

Topographic similarity between the language space and the visual feature space: 0.5490247795754525 (with p=0.0).


In [41]:
zero_lev_dist_card = len([l for l in levs if l==0])
lev_dist_card = len(levs)
print("Number of 0-Levenshtein-distance pairs: {} / {} :: {}%.".format(zero_lev_dist_card, lev_dist_card, float(zero_lev_dist_card)/lev_dist_card*100.0))

threshold = 0.1
thresholded_cossim_dist_card = len([c for c in cossims if abs(c)< threshold])
cossims_dist_card = len(cossims)
print("Number of less-than-{}-cosine-distance pairs: {} / {} :: {}%.".format(threshold, thresholded_cossim_dist_card, cossims_dist_card, float(thresholded_cossim_dist_card)/cossims_dist_card*100.0))


Number of 0-Levenshtein-distance pairs: 67659 / 1979700 :: 3.417639036217609%.
Number of less-than-0.1-cosine-distance pairs: 172 / 1979700 :: 0.008688185078547255%.


## Ambiguity:

In [42]:
def cardinality(data):
    if isinstance(data[0], np.ndarray):
        data_array = np.concatenate([np.expand_dims(d, 0) for d in data], axis=0)
        data_set = np.unique(data_array, axis=0)
    else:
        data_set = set(data)
    return len(data_set)

In [51]:
vocab_size = 5 #np.max(np.concatenate([ np.array([symbol for symbol in sentence]) for sentence in sentences]))+1
print("The size of the vocabulary: {} symbols.".format(vocab_size))

str_s = [ ''.join([chr(97+int(symbol)) for symbol in sentence if symbol<(vocab_size-1)]) for sentence in sentences]
nbr_unique_sentences = cardinality(str_s)
nbr_unique_stimulus = cardinality(features)

print("There are {} unique sentences out of {} different stimuli :: {} %.".format(nbr_unique_sentences, nbr_unique_stimulus, float(nbr_unique_sentences)/nbr_unique_stimulus*100.0))
print("Ambiguity: {} %.".format(float(nbr_unique_stimulus-nbr_unique_sentences)/nbr_unique_stimulus*100.0))

sentence_lengths = [ len([symbol for symbol in sentence if symbol<(vocab_size-1)]) for sentence in sentences]
min_s_length = min(sentence_lengths)
mean_s_length = np.mean(np.array(sentence_lengths))
print("The minimum sentence length is: {}. The average sentence length is: {}.".format(min_s_length, mean_s_length))

The size of the vocabulary: 5 symbols.
There are 615 unique sentences out of 8464 different stimuli :: 7.2660680529300565 %.
Ambiguity: 92.73393194706995 %.
The minimum sentence length is: 1. The average sentence length is: 17.6564.
